In [1]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar"

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName("MyApp2_ReadStreamFromKafka").getOrCreate()

lines = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "192.168.1.123:8097") \
  .option("subscribe", "input") \
  .option("startingOffsets", "earliest") \
  .load()

In [5]:
lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
#Lab 2.1
schema = StructType() \
    .add("event_time", TimestampType()) \
    .add("message", StringType())

parsed = lines.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.event_time", "data.message")

parsed.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- message: string (nullable = true)



In [7]:
words = parsed.select(
   explode(
       split(parsed.message, " ")
   ).alias("word"),
   col("event_time")
)

In [8]:
words.printSchema()

root
 |-- word: string (nullable = false)
 |-- event_time: timestamp (nullable = true)



In [15]:
wordCounts = words.groupBy(
    window("event_Time", "10 minutes", "5 minutes"),
    "word"
).count()

In [16]:
wordCounts.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [ ]:
if False:
    query = wordCounts \
      .writeStream \
      .outputMode("complete") \
      .format("console") \
      .option("truncate", False) \
      .start()
    
    query.awaitTermination()


In [ ]:
'''wordCounts.selectExpr(
    "CAST(word AS STRING) AS key",
    """to_json(named_struct(
        'count', count,
        'timestamp', timestamp
    )) AS value"""
)'''

In [ ]:
#query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
if True:
    query = wordCounts.selectExpr("CAST(word AS STRING) AS key", "CAST(count AS STRING)  AS value") \
        .writeStream \
        .trigger(processingTime='1 seconds') \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "192.168.1.123:8097") \
        .option("topic", "output") \
        .option("checkpointLocation", "checkpoint") \
        .start() \
        
    query.awaitTermination()

In [ ]:
query = wordCounts.selectExpr("CAST(word AS STRING) AS key", "CAST(count AS STRING)  AS value") \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
for q in spark.streams.active:
    q.stop()